<a href="https://colab.research.google.com/gist/mvenouziou/35633b556aca969f7241ae83836c5b4f/style-transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import IPython.display as display
import numpy as np
from PIL import Image

# from tensorflow import keras
# from tensorflow.keras import layers

# import PIL
# import pprint



In [6]:
# Select Images & Model Parameters

SIZE = (224, 224)
CONTENT_WEIGHT = 10000
STYLE_WEIGHT = .01
NUM_ITER = 50
LAYERS_WEIGHTS = {1: 0.2,
                  4: 0.2,
                  7: 0.2,
                  12: 0.2,
                  17: 0.2,
                  18: 0.2,
                  }

In [7]:
# load pre-trained model
# model: ImageNet VGG Very Deep 19

def load_base_model(model='VGG19'):
    # load pre-trained model
    # model: VGG19 or EfficientNetB7

    if model == 'VGG19':
     source_model = \
           tf.keras.applications.VGG19(include_top=False, weights='imagenet', 
                input_tensor=None, input_shape=None, pooling=None, 
                classes=1000, classifier_activation='softmax')
        
    else:  # model == 'EfficientNetB7'  ##### not yet implemented. 
        pass
    """
    ##### raises 'ValueError: A merge layer should be called '
    
      source_model = \
        tf.keras.applications.EfficientNetB7(include_top=False, 
                weights='imagenet', input_tensor=None, input_shape=None,
                pooling=None, classes=1000, classifier_activation='softmax')
    """


    source_model.trainable = False
    return source_model

source_model = load_base_model()

In [8]:
source_model.layers

In [9]:
def view_generated_image(im_array, view=True):
    im_array = im_array.numpy()
    image = \
        tf.keras.preprocessing.image.array_to_img(im_array, data_format=None,
                                                  scale=True, dtype=None)
    if view is True:
        plt.imshow(image)
    return image


def prepare_images(image_content, image_style, size=SIZE):
    """
    # format images as tensors usable by model
    """

    # load images
    content = plt.imread(image_content)
    style = plt.imread(image_style)

    # Resize images
    content = tf.keras.preprocessing.image.smart_resize(content, size)
    style = tf.keras.preprocessing.image.smart_resize(style, size)

    # convert to arrays and standardize
    content = tf.keras.preprocessing.image.img_to_array(content)/255
    style = tf.keras.preprocessing.image.img_to_array(style)/255

    # convert to proper shape: (1, height, width, channels)
    content = tf.expand_dims(content, axis=0)
    style = tf.expand_dims(style, axis=0)

    return content, style


def initialize_generated_image(input_image, noise_rate):
    noise = tf.random.uniform(shape=input_image.shape, minval=-20, maxval=20)
    image = noise * noise_rate + input_image * (1 - noise_rate)
    return image

In [10]:
### Cost Functions
# Difference between input image and generated
# Difference between style image and generated

In [11]:
### Cost Functions
# Difference between input image and generated
# Difference between style image and generated

def style_cost(style_activations, generated_activations, weights):

    # initialize cost
    cost = 0

    for layer in style_activations.keys():
        # select content data
        style = style_activations[layer]
        generated = generated_activations[layer]
        weight = weights[layer]

        # compute scale factor
        height = style.shape[-3]
        width = style.shape[-2]
        num_channels = style.shape[-1]
        scale_factor = 1 / (4 * (height**2) * (width**2) * (num_channels**2))

       # compute cost
        gram_style = gram_matrix(style)
        gram_generated = gram_matrix(generated)

        cost += weight * scale_factor * tf.norm(gram_style - gram_generated, ord=2)**2

    return cost


def gram_matrix(image):
    """
    Computes Gram Matrix
    Parameter:
    image: tensors of shape (px_width, px_height, channels)
    """
    num_channels = image.shape[-1]
    image = tf.reshape(image, [-1, num_channels])
    gram = tf.linalg.matmul(image, image, transpose_a=True, transpose_b=False)
    return gram


def content_cost(image_activations, generated_activations, content_layer):

    # select content data
    image = image_activations[content_layer]
    generated = generated_activations[content_layer]

    # compute scale factor
    height = image.shape[-3]
    width = image.shape[-2]
    num_channels = image.shape[-1]
    scale_factor = 1 / (4 * height * width * num_channels)

    # compute cost
    cost = scale_factor * tf.norm(tensor= image - generated, ord='euclidean')

    return cost

In [12]:

def learn_activations(input_tensor, orig_model, layers_list):
    """
    Conducts forward pass of given model and stores activations
    indicated by layers list.
    """

    # initialize activations container
    activations_computed = dict()
    num_layers = len(orig_model.layers)  # number of activations to record
    layers_list = [1] + layers_list  # adjustment for better looping

    # forward pass

    x = orig_model.layers[1](input_tensor)

    for k in range(1, len(layers_list)):
        for i in range(layers_list[k - 1] + 1, layers_list[k] + 1):
            x = orig_model.layers[i](x)
        # store activation
        activations_computed[layers_list[k]] = x

    #for i in range(layers_list[-1] + 1, num_layers):  # final step
    #    x = orig_model.layers[i](x)
    #activations_computed['output'] = x

    return activations_computed


In [13]:
def train_generated_image(init_generated_image, image_activations, 
                          style_activations, cnn_model, layers_dict, 
                          content_weight, style_weight, num_iterations):

    computed_images_dict = dict()

    # get parameters
    layers = list(layers_dict.keys())  # lists of CNN layer numbers
    content_layer = list(layers_dict.keys())[-1]

    # initialize variables
    learned_im = tf.Variable(init_generated_image)

    # set optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.9, 
                                         beta_2=0.999, epsilon=1e-1, 
                                         amsgrad=False, name='Adam')

    for iter in range(num_iterations):
        with tf.GradientTape() as tape:
            tape.watch(learned_im)

            def loss_function():
                generated_activations = \
                    learn_activations(learned_im, cnn_model, layers)
                # compute loss
                loss = content_weight * content_cost(image_activations, 
                                                     generated_activations, 
                                                     content_layer) + \
                        style_weight * style_cost(style_activations, 
                                                  generated_activations, 
                                                  layers_dict)
                return loss

        # apply gradients
        optimizer.minimize(loss=loss_function, var_list=[learned_im])
        learned_im = \
            tf.clip_by_value(learned_im, clip_value_min=0.0, clip_value_max=1.0)
        learned_im = tf.Variable(init_generated_image)

        # give status updates
        if iter % 100 == 0:
            print("iteration:", iter)
            print("loss:", loss_function().numpy())
            # save image to dictionary
            computed_images_dict[iter] = learned_im
            
            if iter % 150 == 0:
                learned_image = tf.convert_to_tensor(learned_im * 255)
                learned_image = tf.squeeze(learned_image)
                display.clear_output(wait=True)
                display.display(view_generated_image(learned_image, False))
                
    print("Completed.")
    print("loss:", loss_function().numpy())

    learned_image = tf.convert_to_tensor(learned_im * 255)
    learned_image = tf.squeeze(learned_image)

    return learned_image, computed_images_dict


In [14]:
def style_transfer(image_content, image_style, layers_dict=LAYERS_WEIGHTS, *kargs):

    # prepare images
    image_tensor, style_tensor = prepare_images(image_content, image_style)

    init_generated_image = initialize_generated_image(image_tensor, noise_rate=.01)
    init_generated_image = tf.convert_to_tensor(init_generated_image)

    # prepare model parameters
    image_model = load_base_model()  # pre-trained image processing CNN
    layers = list(layers_dict.keys())  # lists CNN layer numbers for transfer
    image_activations = learn_activations(image_tensor, image_model, layers)
    style_activations = learn_activations(style_tensor, image_model, layers)

    # style transfer model
    G_new, computed_images_dict = \
        train_generated_image(init_generated_image, image_activations, 
                              style_activations, image_model, layers_dict, 
                              content_weight=CONTENT_WEIGHT, 
                              style_weight=STYLE_WEIGHT, num_iterations=NUM_ITER)

    return G_new, computed_images_dict

In [74]:


import urllib
import requests
from PIL import Image
from io import BytesIO


def linkFetch(unsplash_photo_id):
    url = 'https://images.unsplash.com/' + unsplash_photo_id
    headers = {'authorization': 'Client-ID tKLJiokp1I7MXnQUwh_UWbkHYIXgxPSTBaT7m2ox1xk'}
    
    response = requests.get(url, headers=headers)
    print(response.text)
    #data = response.json()["urls"]["raw"]
    return data

true_image_file = linkFetch('photo-1556796189-d37c540abddb')
#true_image = requests.get(true_image_file)
#style_image_file = linkFetch('_QxzSVWesm0')
#style_image = requests.get(style_image_file)

#true_image = Image.open(BytesIO(true_image.content))
#style_image = Image.open(BytesIO(style_image.content))
#style_image.show()



"""
with open('tokens.json', 'r') as f:
    data = json.load(f)client_id = data['client_id']
client_secret = data['client_secret']redirect_uri = ""
code = ""

auth = Auth(client_id, client_secret, redirect_uri, code=code)
api = Api(auth)


def url_to_image(url):
	photo = urllib.request.urlopen(url)
	image = Image.open(photo)
	return image

true_image_file = 'https://images.unsplash.com/photo-1556796189-d37c540abddb?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=564&q=80'
style_image_file = 'https://unsplash.com/photos/_QxzSVWesm0/download?force=true&w=640'
"""

#true_image  = tf.keras.preprocessing.image.img_to_array(true_image_file)
#style_image  = tf.keras.preprocessing.image.img_to_array(style_image_file)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



NameError: ignored

In [64]:
style_image_file

'https://unsplash.com/photos/_QxzSVWesm0/download?force=true&w=640'

In [66]:
# select images and run program


#-- for custom images on Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


# Public image datasets from tensorflow-datasets
#import tensorflow_datasets as tfds
#style_image_library = tfds.load('shapes3d', try_gcs=True, download=False)
#style_image = style_image_library.take(1)['image']

#true_image_library = tfds.load('vgg_face2', try_gcs=True, download=False)
#true_image = style_image_library.take(1)['image']
# assert isinstance(mnist_train, tf.data.Dataset)


# Run program
image, itermediary_images = \
    style_transfer(img, img, layers_dict=LAYERS_WEIGHTS)

ValueError: ignored